In [2]:
# -*- coding=utf-8 -*-

#对moldata.csv，并进行预处理，删除无效信息
#将数据分为“smiles”“pce”两部分存储在列表data[]中
import pandas as pd
from rdkit import Chem

path = 'F:\Jupyter Notebook\materials genes\data\moldata.csv'
data = pd.read_csv(path)
smiles = data['SMILES_str'][0]
smiles
with open(path, 'r', encoding='utf-8') as f:
    o_smiles = 1
    o_pce = 4
    data = []
    row = []
    count = 0
#     for line in f.readlines():
#         if count == 0:
#             data = line.split(',')
#             row = [eval(line[o_smiles]), eval(eval(line[o_pce]))]
#             data.append(row)
#             count += 1
#         else:
#             break
    for line in f.readlines():
        line = line.split(',')
        if count == 0:
            count += 1
            continue
        else:
            if eval(eval(line[o_pce])) == 0:
                continue
            elif type(Chem.MolFromSmiles(line[o_smiles])) is None:
                continue
            else:
                row = [eval(line[o_smiles]), eval(eval(line[o_pce]))]
                data.append(row)
                count += 1
f.close()
print(data[0])
print(type(data[0][0]))

['C1C=CC=C1c1cc2[se]c3c4occc4c4nsnc4c3c2cn1', 5.16195320211971]
<class 'str'>


In [61]:
#将数据分为“smiles”“pce”两部分存储在列表data[]中
# import pandas as pd 

# path = 'F:\Jupyter Notebook\materials genes\data\moldata.csv'
# data = pd.read_csv(path)
# smiles = data['SMILES_str'][1]

atom_dict = defaultdict(lambda: len(atom_dict))
bond_dict = defaultdict(lambda: len(bond_dict))
fingerprint_dict = defaultdict(lambda: (len(fingerprint_dict)))
edge_dict = defaultdict(lambda: len(edge_dict))

smiles = Chem.MolFromSmiles(smiles)

# atomses = []
    # 取出一个分子中的所有原子
# for a in smiles.GetAtoms():
#     atomses.append(a.GetSymbol())
    
# for a in smiles.GetAromaticAtoms():
#     i = a.GetIdx()
#     atomses[i] = (atomses[i], 'aromatic')
# atomses = [a.GetSymbol() for a in smiles.GetAtoms()]
# atomses
           
# 调用get_atoms()函数，生成该分子的原子字典
atoms = get_atoms(smiles)
# 调用get_ijbonddict()函数，生成该分子的成键字典
i_jbond_dict = get_ijbonddict(smiles)
# 调用extract_fingerprints()函数，生成半径为1的分子指纹
fingerprint = extract_fingerprints(atoms, i_jbond_dict, 1)
# 使用BFS算法排列分子片段
graph = get_mol_graph(smiles)
atom_sequence = get_bfs_graph(graph)
new_fingerprint = get_new_sequence_fingerprint(atom_sequence, fingerprint)
new_fingerprint

array([ 0,  1,  2,  1,  1,  3,  4, 11,  5, 12, 11,  6, 10, 13, 12,  5,  7,
       10,  8,  4, 10,  9,  4,  4,  4])

In [10]:
#Z-score标准化
from sklearn import preprocessing
import numpy as np
count = 0
smiles = 0
pce = 1
numlist = []
# print(data[0])
# print(type(eval(data[0][pce])))
for item in data:
    numlist.append(item[pce])
# 导入相关数据，将其转化为数组
s_numlist = np.array(numlist)
# 将数据标准化
s_numlist_scaled = preprocessing.scale(s_numlist)

# 标准化后数据的均值
print(s_numlist_scaled.mean())
# 标准化后数据的标准差
print(s_numlist_scaled.std())
# 标准化前的数据集
# print(s_numlist)
# 标准化后的数据集
# print(s_numlist_scaled)

# 将标准化后的数据写入原data[]数据集中
for pce_scaled in s_numlist_scaled:
    data[count][pce] = pce_scaled
    count += 1
print(data[0])

# 数据预处理部分结束

-9.804420244587277e-16
0.9999999999999996
['C1C=CC=C1c1cc2[se]c3c4occc4c4nsnc4c3c2cn1', 0.43296375307150325]


In [58]:
# 获取分子片段池及提取分子特征
import pandas as pd
import numpy as np
from rdkit.Chem import Draw
from collections import defaultdict
import os
import pickle
import sys
import networkx as nx
from rdkit import Chem
import matplotlib.pyplot as plt

# 定义一些函数用于获取分子片段池和提取分子特征

# 输入mol格式的分子，生成该分子的原子字典
def get_atoms(mol):
    atoms = []
    atom = []
    # 取出一个分子中的所有原子
    for a in mol.GetAtoms():
        atoms.append(a.GetSymbol())
    # 将分子中的芳香性原子标注出来
    for a in mol.GetAromaticAtoms():
        i = a.GetIdx()
        atoms[i] = (atoms[i], 'aromatic')
    for a in atoms:
        atom.append(atom_dict[a])
    # 返回每个原子在atom_dict中对应的value
    return np.array(atom)

def get_ijbonddict(mol):
    # 在ijbond_dict列表中，为每个原子生成记录与之相连的键的字典
    ijbond_dict = defaultdict(lambda:[])
    # 分析分子中的化学键
    for b in mol.GetBonds():
        i = b.GetBeginAtomIdx()
        j = b.GetEndAtomIdx()
        # 记录键的类型
        bond = bond_dict[str(b.GetBondType())]
        # 为每个原子形成与之相连的键的列表
        # 以原子编号为key，与之相连的(原子编号, 二者之间的键)为value，生成字典
        ijbond_dict[i].append((j, bond))
        ijbond_dict[j].append((i, bond))
    # 返回每个原子的成键情况字典
    # ijbond_dict = {0:[(1, bond_dict[bond])], ...}
    return ijbond_dict

def extract_fingerprints(atoms, ijbond_dict, radius):
    fingerprints = []
    if (len(atoms) == 1) or (radius == 0):
        for a in atoms:
            fingerprints.append(fingerprint_dict[a])
    else:
        nodes = atoms
        ij_jedge_dict = ijbond_dict
        for r in range(radius):
            fingerprints = []
            neighbors = []
            # 取出成键情况字典的所有值，即每个原子上连接的键和原子
            for i, j_edge in ij_jedge_dict.items():
                # 取出键上连接的原子编号和键
                for j, edge in j_edge:
                    neighbors.append((nodes[j], edge))
                fingerprint = (nodes[i], tuple(sorted(neighbors)))
                fingerprints.append(fingerprint_dict[fingerprint])
                neighbors.clear()
            nodes = fingerprints
            _ij_jedge_dict = defaultdict(lambda:[])
            for i, j_edge in ij_jedge_dict.items():
                for j, edge in j_edge:
                    both_side = tuple(sorted((nodes[i], nodes[j])))
                    egde = edge_dict[(both_side, edge)]
                    _ij_jedge_dict[i].append((j, edge))
            # 统计片段类型，反应中心原子(atom_dict中的index)与相连原子(atom_dict中的index)形成的键(bond_dict中的index)的情况
            ij_edge_dict = _ij_jedge_dict
    #返回按照smiles中的index排列的fingerprint_dict中对应的片段value的指纹
    return np.array(fingerprints)

# 构建分子图，为BFS遍历图形做准备
def get_mol_graph(mol):
    ij_bond = []
    # 创建空的简单图
    mol_graph = nx.Graph()
    # 绘制各边
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        ij_bond.append((i, j))
        mol_graph.add_edge(i, j)
    # 返回分子图
    return mol_graph

# 用BFS搜索方法遍历图形
def get_bfs_graph(graph):
    # 按照BFS遍历路径对smiles进行排序
    bfs_graph = nx.bfs_tree(graph, source = 0)
    bfs_nodes = list(bfs_graph)
    # 返回经BFS遍历排序后的smiles的index
    return bfs_nodes

# 生成新的分子指纹
def get_new_sequence_fingerprint(nodes_sequence, fingerprint):
    new_sequence_fingerprint = []
    for node in nodes_sequence:
        new_sequence_fingerprint.append(fingerprint[node])
    new_sequence_fingerprint = np.array(new_sequence_fingerprint)
    # 在fingerprint中按照BFS遍历后的排序分别取出相应的片段
    return new_sequence_fingerprint

# mol = Chem.MolFromSmiles(data[0][1])
# print(type(data[0][1]))
# print(type(mol))

In [15]:
# 开始对数据进行处理
from collections import defaultdict
from rdkit import Chem

# 定义一些全局变量
smiles_num = 0
pce_num = 1
count = 0
cal_fp_list, cal_pce_list = [], []
atom_dict = defaultdict(lambda: len(atom_dict))
bond_dict = defaultdict(lambda: len(bond_dict))
fingerprint_dict = defaultdict(lambda: (len(fingerprint_dict)))
edge_dict = defaultdict(lambda: len(edge_dict))

for item in data:
    smiles, pce = item[smiles_num], item[pce_num]
    # 将smiles字符串改写为mol格式
    smiles = Chem.MolFromSmiles(smiles)
    # 调用get_atoms()函数，生成该分子的原子字典
    atoms = get_atoms(smiles)
    # 调用get_ijbonddict()函数，生成该分子的成键字典
    i_jbond_dict = get_ijbonddict(smiles)
    # 调用extract_fingerprints()函数，生成半径为1的分子指纹
    fingerprint = extract_fingerprints(atoms, i_jbond_dict, 1)
    # 使用BFS算法排列分子片段
    graph = get_mol_graph(smiles)
    atom_sequence = get_bfs_graph(graph)
    # 调用get_new_sequence_fingerprints()函数，根据BFS排列后的分子片段顺序排列生成new_fingerprint
    new_fingerprint = get_new_sequence_fingerprint(atom_sequence, fingerprint)
    # 列表保存处理后的分子指纹和pce值
    cal_fp_list.append(new_fingerprint)
    cal_pce_list.append([float(pce)])

# 用分子指纹固定每个分子的片段长度
fragment_num = len(fingerprint_dict)
# 不同长度分子片段的种类数
print("分子片段种类数:{}".format(fragment_num))

# 统计分子片段长度
atom_number = []
for fp in cal_fp_list:
    atom_number.append(len(fp))
mol_len = max(atom_number)
print("分子含片段数:({}, {})".format(min(atom_number), mol_len))

# 将不同长度的分子片段指纹补齐至长度一致，保存在cal_nfp_list[]中
cal_nfp_list = []
for fp in cal_fp_list:
    if len(fp) < mol_len + 1:
        nfp = np.append(fp, (mol_len + 1 - len(fp)) * ([fragment_num + 1]))
        cal_nfp_list.append(nfp)

OK1


IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed